In [24]:
from os import name
#  read our dataset 
import pandas as pd
# read from drive
messages = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',encoding = "latin-1")

messages.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [25]:
messages.rename(columns = {'v1':'label', 'v2':'message'}, inplace = True)

In [26]:
messages = messages[['label','message']].copy()

In [27]:
messages.shape

(5572, 2)

In [28]:
# data cleaning || data preprocessing
import re,nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
    # Apply lemmatization 
from nltk.stem import WordNetLemmatizer
lemmtizer = WordNetLemmatizer()

In [30]:
corpus = []
y =[]
for i in range(len(messages)):
    message = re.sub('[^a-zA-Z0-9]',' ',messages['message'][i])
    message = message.lower()
    message = message.split()
    # apply lemmatization
    message = [lemmtizer.lemmatize(word)  for word in message ]
    message = ' '.join(message)
    corpus.append(message)
print(len(corpus))

5572


In [31]:
# messages.drop(messages[messages['message'] == ''].index ).shape

In [32]:
len(corpus)

5572

In [33]:
#  Get tokens from documents
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [34]:
words =[]
for doc in corpus:
#     print(f'befor: {sent}')
    sent_token = sent_tokenize(doc)
#     print(sent_token)
#     print(sent_token[0])
    for sent in sent_token:
        words.append(simple_preprocess(sent,max_len=50))
print(len(words))

5570


In [35]:
list1 = [[l,i,j,k] for l,i,j,k in zip(range(len(corpus)) ,list(map(len,corpus)),corpus, messages['message']) if i<1]
print(list1)
for i in list1:
  print(messages['message'][i[0]],messages['label'][i[0]],corpus[i[0]],len(corpus[i[0]]))

[[3374, 0, '', ':) '], [4822, 0, '', ':-) :-)']]
:)  ham  0
:-) :-) ham  0


In [36]:
# Let's try to create our avgworde2vec
import gensim
model = gensim.models.Word2Vec(words,vector_size=200,epochs=3)
# model = gensim.models.Word2Vec(words,iter=3, size=200) 

In [37]:
# Get All the Vocabulary (Unique Words)
model.wv.index_to_key

['you',
 'to',
 'the',
 'and',
 'it',
 'in',
 'is',
 'me',
 'my',
 'for',
 'your',
 'call',
 'of',
 'that',
 'have',
 'on',
 'now',
 'are',
 'can',
 'so',
 'but',
 'not',
 'or',
 'we',
 'do',
 'get',
 'at',
 'be',
 'if',
 'will',
 'ur',
 'with',
 'no',
 'just',
 'this',
 'gt',
 'lt',
 'go',
 'how',
 'up',
 'when',
 'ok',
 'what',
 'free',
 'day',
 'from',
 'all',
 'out',
 'know',
 'll',
 'come',
 'like',
 'good',
 'time',
 'then',
 'am',
 'got',
 'wa',
 'there',
 'he',
 'text',
 'only',
 'love',
 'want',
 'send',
 'txt',
 'need',
 'one',
 'today',
 'going',
 'by',
 'home',
 'don',
 'about',
 'stop',
 'she',
 'lor',
 'sorry',
 'see',
 'still',
 'mobile',
 'take',
 'back',
 'da',
 'our',
 'reply',
 'think',
 'dont',
 'tell',
 'week',
 'phone',
 'hi',
 'new',
 'they',
 'later',
 'pls',
 'her',
 'any',
 'ha',
 'please',
 'co',
 'did',
 'been',
 'msg',
 'an',
 'some',
 'dear',
 'make',
 'here',
 'message',
 'who',
 'night',
 'say',
 'well',
 're',
 'min',
 'where',
 'thing',
 'much',
 'grea

In [38]:
# model.wv.similar_by_word('good')

In [39]:
model.wv['good'].shape

(200,)

In [40]:
import numpy as np

In [41]:
print(gensim.__version__)

4.0.1


In [42]:
def avgWord2Vec(doc):
    # return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [43]:
from tqdm import tqdm

In [44]:
X = []
for i in tqdm(range(len(words))):
    X.append(avgWord2Vec(words[i]))

  0%|          | 0/5570 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5570/5570 [00:01<00:00, 5330.22it/s]


In [45]:
len(X)

5570

In [46]:
X[1]

array([-0.03997364, -0.0442351 , -0.03273924,  0.10623898,  0.19074847,
       -0.09081425, -0.00672596,  0.27070114, -0.03011958,  0.06482121,
       -0.11251313, -0.1342547 ,  0.04209808,  0.10885122,  0.0099213 ,
       -0.09410373, -0.02427218,  0.04335858,  0.06265762, -0.22190306,
        0.03085338, -0.05205325, -0.00226171,  0.03885442,  0.00936877,
       -0.08030148, -0.01900613, -0.09475471, -0.15557411,  0.07277711,
        0.1979522 ,  0.054182  ,  0.06453366, -0.00453279, -0.00671564,
        0.06082278,  0.12121403, -0.06893727, -0.08096845, -0.19202562,
       -0.10878532, -0.04002509, -0.08000018,  0.09736969,  0.2539693 ,
        0.00981459, -0.04153935, -0.08331265,  0.12297227,  0.13020346,
        0.05299572, -0.0565481 , -0.07352951, -0.10212493,  0.04942247,
       -0.0833343 ,  0.00806718, -0.12077931, -0.18924367, -0.00338973,
       -0.05814825, -0.03765036,  0.0092141 , -0.02722329, -0.24342605,
        0.06075636, -0.02111554,  0.22148006, -0.1221349 ,  0.19

In [47]:
X[1].shape

(200,)

In [48]:
print(len(X))
print(X[0].shape)
print(messages.shape)
#   As we see here there is a different between massages shape and X shape
#   we need to fix it

5570
(200,)
(5572, 2)


In [49]:
x_array = np.array(X)
print(x_array.shape)
# print(X.shape)

(5570,)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [50]:
y = messages[list(map(lambda x: len(x)> 0 , corpus))]
# print(y)
y = pd.get_dummies(y['label'])
# print(y.shape)
y = y.iloc[:,0].values
print(y.shape)
print(y)
# print(messages['label'])

(5570,)
[1 1 0 ... 1 1 1]


In [51]:
print(len(X))
print(X[0].shape)
print(y.shape)

5570
(200,)
(5570,)


In [52]:
# create one hot encode to each feature in vec 
# ex
print(X[0].shape)
print(X[0].reshape(1,-1).shape)

(200,)
(1, 200)


In [53]:
# # create df to independent features
# df = pd.DataFrame()
# for i in range(len(X)):
# #     df = df.append(pd.DataFrame(X[i].reshape(1,-1)),ignore_index=True)
#     df = pd.concat([df,pd.DataFrame(X[i].reshape(1,-1))],ignore_index=True)
# print(df.shape)
# df.head()

In [54]:
# create df to independent features
df = pd.DataFrame()
for i in range(len(X)):
    df = pd.concat([df,pd.DataFrame(X[i].reshape(1,-1))],ignore_index=True)
print(df.shape)
df.head()

(5570, 200)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.055663,-0.057004,-0.042113,0.144506,0.256378,-0.124145,-0.010961,0.363167,-0.041259,0.089024,...,0.119404,-0.049663,-0.182903,-0.150295,0.114270,0.067335,0.091746,-0.195583,0.001686,-0.085051
1,-0.039974,-0.044235,-0.032739,0.106239,0.190748,-0.090814,-0.006726,0.270701,-0.030120,0.064821,...,0.091796,-0.037015,-0.134414,-0.114688,0.083752,0.050660,0.067548,-0.145681,0.004055,-0.063627
2,-0.058357,-0.059174,-0.045558,0.154555,0.270945,-0.131335,-0.011062,0.382788,-0.044126,0.091099,...,0.128547,-0.055769,-0.193998,-0.159018,0.121132,0.071582,0.097035,-0.207062,0.001440,-0.090212
3,-0.077800,-0.079806,-0.060098,0.198092,0.347199,-0.168589,-0.014003,0.490593,-0.054752,0.119364,...,0.166312,-0.068398,-0.247534,-0.206742,0.155990,0.093394,0.125721,-0.265432,0.006190,-0.115830
4,-0.067827,-0.068168,-0.052107,0.170659,0.304916,-0.146947,-0.013800,0.431287,-0.050712,0.106929,...,0.143505,-0.057824,-0.216024,-0.179540,0.138087,0.082005,0.109046,-0.232332,0.003591,-0.101260


In [55]:
df['Output'] = y
df.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,Output
0,-0.055663,-0.057004,-0.042113,0.144506,0.256378,-0.124145,-0.010961,0.363167,-0.041259,0.089024,...,-0.049663,-0.182903,-0.150295,0.114270,0.067335,0.091746,-0.195583,0.001686,-0.085051,1
1,-0.039974,-0.044235,-0.032739,0.106239,0.190748,-0.090814,-0.006726,0.270701,-0.030120,0.064821,...,-0.037015,-0.134414,-0.114688,0.083752,0.050660,0.067548,-0.145681,0.004055,-0.063627,1
2,-0.058357,-0.059174,-0.045558,0.154555,0.270945,-0.131335,-0.011062,0.382788,-0.044126,0.091099,...,-0.055769,-0.193998,-0.159018,0.121132,0.071582,0.097035,-0.207062,0.001440,-0.090212,0
3,-0.077800,-0.079806,-0.060098,0.198092,0.347199,-0.168589,-0.014003,0.490593,-0.054752,0.119364,...,-0.068398,-0.247534,-0.206742,0.155990,0.093394,0.125721,-0.265432,0.006190,-0.115830,1
4,-0.067827,-0.068168,-0.052107,0.170659,0.304916,-0.146947,-0.013800,0.431287,-0.050712,0.106929,...,-0.057824,-0.216024,-0.179540,0.138087,0.082005,0.109046,-0.232332,0.003591,-0.101260,1


In [56]:
df.isnull().sum()

0         13
1         13
2         13
3         13
4         13
          ..
196       13
197       13
198       13
199       13
Output     0
Length: 201, dtype: int64

In [57]:
df.dropna(inplace=True)
df.isnull().sum()

0         0
1         0
2         0
3         0
4         0
         ..
196       0
197       0
198       0
199       0
Output    0
Length: 201, dtype: int64

In [58]:
df.shape

(5557, 201)

In [59]:
model.wv.vector_size

200

In [60]:
model.get_latest_training_loss

<bound method Word2Vec.get_latest_training_loss of <gensim.models.word2vec.Word2Vec object at 0x7f87dc1860d0>>

In [61]:
# df.to_csv('datasets/v1.csv',index=False)
# print(df.shape)
# data = pd.read_csv('datasets/v1.csv',index_col=False)
# print(data.shape)
# data.head()

In [62]:
X = df.drop(['Output'],axis=1)

In [63]:
print(X.shape)
X.head()

(5557, 200)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.055663,-0.057004,-0.042113,0.144506,0.256378,-0.124145,-0.010961,0.363167,-0.041259,0.089024,...,0.119404,-0.049663,-0.182903,-0.150295,0.114270,0.067335,0.091746,-0.195583,0.001686,-0.085051
1,-0.039974,-0.044235,-0.032739,0.106239,0.190748,-0.090814,-0.006726,0.270701,-0.030120,0.064821,...,0.091796,-0.037015,-0.134414,-0.114688,0.083752,0.050660,0.067548,-0.145681,0.004055,-0.063627
2,-0.058357,-0.059174,-0.045558,0.154555,0.270945,-0.131335,-0.011062,0.382788,-0.044126,0.091099,...,0.128547,-0.055769,-0.193998,-0.159018,0.121132,0.071582,0.097035,-0.207062,0.001440,-0.090212
3,-0.077800,-0.079806,-0.060098,0.198092,0.347199,-0.168589,-0.014003,0.490593,-0.054752,0.119364,...,0.166312,-0.068398,-0.247534,-0.206742,0.155990,0.093394,0.125721,-0.265432,0.006190,-0.115830
4,-0.067827,-0.068168,-0.052107,0.170659,0.304916,-0.146947,-0.013800,0.431287,-0.050712,0.106929,...,0.143505,-0.057824,-0.216024,-0.179540,0.138087,0.082005,0.109046,-0.232332,0.003591,-0.101260


In [64]:
Y = df['Output']
print(Y.shape)
Y.head()

(5557,)


0    1
1    1
2    0
3    1
4    1
Name: Output, dtype: uint8

In [65]:
# split data to trai and test
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=True)

In [66]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
4056,-0.069025,-0.069298,-0.056005,0.180415,0.317109,-0.154015,-0.013505,0.454216,-0.054259,0.109385,...,0.151035,-0.062106,-0.226544,-0.189276,0.142932,0.085248,0.113409,-0.243208,0.005297,-0.105813
3112,-0.077138,-0.080673,-0.059492,0.200414,0.353873,-0.173621,-0.014647,0.501634,-0.057224,0.123505,...,0.166213,-0.068920,-0.250901,-0.208780,0.159905,0.091544,0.129380,-0.270409,0.003833,-0.115951
1046,-0.066520,-0.067137,-0.051077,0.169859,0.300210,-0.146495,-0.013531,0.428219,-0.049981,0.104618,...,0.143616,-0.057627,-0.215059,-0.179134,0.136744,0.081355,0.107287,-0.228877,0.002098,-0.101021
5101,-0.077643,-0.079144,-0.061604,0.200509,0.356224,-0.174716,-0.012399,0.505588,-0.058340,0.121803,...,0.167551,-0.068968,-0.254190,-0.209916,0.160507,0.093570,0.126044,-0.271546,0.003090,-0.118335
3642,-0.069145,-0.066056,-0.052625,0.175889,0.307487,-0.155752,-0.013504,0.442007,-0.052301,0.107843,...,0.143342,-0.059360,-0.220971,-0.185806,0.138902,0.082847,0.110873,-0.236678,0.002270,-0.102873


In [67]:
Y_train.head()

4056    1
3112    1
1046    1
5101    1
3642    1
Name: Output, dtype: uint8

In [68]:
# using RandomForestClassifier to be our model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()

In [69]:
classifier.fit(X_train,Y_train)

RandomForestClassifier()

In [70]:
Y_pred = classifier.predict(X_test)

In [71]:
#  Get accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test,Y_pred))

0.954136690647482


In [72]:
# Get f1 score
from sklearn.metrics import f1_score
print(f'F1 score of AvgWord2Vec : {f1_score(Y_test,Y_pred)} ')

F1 score of AvgWord2Vec : 0.9736706246773361 


In [73]:
# Get classification report
from sklearn.metrics import classification_report
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.93      0.74      0.82       160
           1       0.96      0.99      0.97       952

    accuracy                           0.95      1112
   macro avg       0.94      0.86      0.90      1112
weighted avg       0.95      0.95      0.95      1112

